## Here is the code to get the paper citations

In [53]:
import bs4 as bs
import urllib.request
import re as re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
#     def get_full_text(self):
#         self.html = urllib.request.urlopen(self.paper_url)# grab html
#         self.soup = bs.BeautifulSoup(self.html,'lxml')# trun pretty 
#         ## extract
#         self.citations = self.soup.find_all('div', attrs={'class':'References'})
#         self.cite_flag = True;
#         if self.citations is None:
#             print('cant get citations for ',self.paper_url) 
#         elif not self.citations: # NN
#             self.citations = self.soup.find('ol', {'class':'references'}).select('li[id]') # get citations
#             if self.citations is None:
#                 print('cant get citations for ',self.paper_url) 
#             elif not a.citations:
#                 print('cant get citations for ',self.paper_url)

In [ ]:

# links_output = []
# for link in a.soup.find_all('a'):
#     links_output.append(link.get('href'))
    
# for i in range(0,len(links_output)): 
#     if links_output[i] is None:
#         continue
#     else:
#         if '.full' in links_output[i]:
#             if 'http:' in links_output[i]:
#                 master_link = links_output[i]
#                 break

In [54]:
# need to get j neuroscience to work better
class nerdy:
    def __init__(self, paper_url):
        if 'http://dx' in paper_url:
            print('a')
            self.paper_url = paper_url
        elif 'http' in paper_url:
            print('b')
            self.paper_url = paper_url
        else:
            print('c')
            self.paper_url = 'http://dx.doi.org/' + paper_url

        
    def elsevier_ref(self):
        prefix = 'https://api.elsevier.com/content/abstract/doi/'
        suffix = '?apiKey=40acd4c83833cb8ead38b50834823fd9&view=REF'
        doi = self.paper_url

        self.link = prefix+doi+suffix
        self.html = urllib.request.urlopen(self.link)# grab html
        self.soup = bs.BeautifulSoup(self.html,'xml')# trun pretty 
        
        self.citations =[]
        for doi in self.soup.find_all('doi'):
            self.citations.append(doi.contents[0])
        
        self.random_ten = np.random.choice(self.citations,size = 10)
                        
    def get_full_text(self):
        # added error handeling
        req = urllib.request.Request(self.paper_url)
        try: urllib.request.urlopen(req)
        except urllib.error.URLError as e:
            print('link ' + e.reason)
            return
        else:
            self.html = urllib.request.urlopen(self.paper_url)
        
#         self.html = urllib.request.urlopen(self.paper_url)# grab html
        self.soup = bs.BeautifulSoup(self.html,'lxml')# trun pretty 
        ## extract
        ## auto detect journal
        if "Nature" in self.soup.title.get_text():
            print('Nature Journal')
            self.journal = 'Nature'
            self.citations = self.soup.find('ol', {'class':'references'}).select('li[id]')
            
            counter = 0
            prefix = ''#'http://dx.doi.org/'
            links_output = []
            for i in range(len(self.citations[0:])):
                if self.citations[i].a is None: # if link doesnt exist 
                    counter = counter + 1
                    print(i, 'cant find link')
                elif len(self.citations[i].a['href']) > 90: # if the link is really long 
                    print(i, 'long link')
#                     links_output.append(re.sub("http://dx.doi.org/", '' ,self.citations[i].a['href']))
                elif 'http://dx.doi.org/' in self.citations[i].a['href']: # normal
                    links_output.append(re.sub("http://dx.doi.org/", '' ,self.citations[i].a['href']))
                else: # if its short add prefix
                    links_output.append(re.sub("/doifinder/", '' ,self.citations[i].a['href']))
                    
            ## save em to class
            self.links_output = links_output
            self.missing_links = counter
            self.random_ten = np.random.choice(self.links_output,size = 10)
            
        elif "ScienceDirect" in self.soup.title.get_text(): 
            print(self.soup.title.get_text())
            
            prefix = 'https://api.elsevier.com/content/abstract/doi/'
            suffix = '?apiKey=40acd4c83833cb8ead38b50834823fd9&view=REF'
            doi = self.paper_url

            self.link = prefix+doi+suffix
            self.html = urllib.request.urlopen(self.link)# grab html
            self.soup = bs.BeautifulSoup(self.html,'xml')# trun pretty 

            self.citations =[]
            for doi in self.soup.find_all('doi'):
                self.citations.append(doi.contents[0])

            self.random_ten = np.random.choice(self.citations,size = 10)
            
        elif "Science" in self.soup.title.get_text(): # science
            print(self.soup.title.get_text())
            
            print('getting full text','Science Journal')
            self.journal = 'Science'
            self.soup = bs.BeautifulSoup(self.html,'lxml')#        
            self.citations = self.soup.find_all('div', attrs={'class':'References'})
            if self.citations is None:
                print('actually abstract getting really')
                links_output = []
                for link in self.soup.find_all('a'):
                    links_output.append(link.get('href'))

                for i in range(0,len(links_output)): 
                    if links_output[i] is None:
                        continue
                    else:
                        if '.full' in links_output[i]:
                            if 'http:' in links_output[i]:
                                self.master_link = links_output[i]
                                break
                self.html = urllib.request.urlopen(self.master_link)# grab html
                self.soup = bs.BeautifulSoup(self.html,'lxml')#        
                self.citations = self.soup.find_all('div', attrs={'class':'References'})
                self.random_ten = np.random.choice(self.citations,size = 10)
            else:
                print(self.citations)
                self.random_ten = np.random.choice(self.citations,size = 10)
        
    def get_links(self):    
        ## get links
        counter = 0
        prefix = ''#'http://dx.doi.org/'
        links_output = [] # create output 
        if self.cite_flag == True:
            for i in range(len(self.citations[0:])):
                if len(self.citations[i].find_all('a')) == 4:
                    links_output.append(self.citations[i].find_all('a')[3]['href'])
                elif len(self.citations[i].find_all('a')) < 2 :
                    counter = counter + 1
                    print(i,'cant find link')
                elif 'http://dx.doi.org/' in self.citations[i].find_all('a')[1]['href']:
                    links_output.append(self.citations[i].find_all('a')[1]['href'])
                else:
                    counter = counter + 1
                    print(i,'cant find link')

        else: ## v1 
            for i in range(len(self.citations[0:])):
                if self.citations[i].a is None: # if link doesnt exist 
                    counter = counter + 1
                    print(i, 'cant find link')
                elif len(self.citations[i].a['href']) > 120: # if the link is really long 
                    links_output.append(re.sub("http://dx.doi.org/", '' ,self.citations[i].a['href']))
                elif 'http://dx.doi.org/' in self.citations[i].a['href']: # normal
                    links_output.append(self.citations[i].a['href'])
                else: # if its short add prefix
                    links_output.append(prefix + re.sub("/doifinder/", '' ,self.citations[i].a['href']))
                    
        ## save em to class
        self.links_output = links_output
        self.missing_links = counter
        self.random_ten = np.random.choice(self.links_output,size = 10)

In [56]:
a = nerdy('http://dx.doi.org/10.1038/35049064')
a.get_full_text()

a
link Unauthorized


In [24]:
a.random_ten

array(['10.1038/33396', '10.1037//0735-7044.112.6.1291',
       '10.1016/S0165-0173(98)00018-6', '10.1016/S0896-6273(00)80214-7',
       '10.1073/pnas.93.24.13494', '10.1016/S0896-6273(00)80214-7',
       '10.1006/brcg.1994.1040', '10.1037//0735-7044.114.3.459',
       '10.1006/nimg.1995.1038', '10.1038/33396'], 
      dtype='<U63')

In [49]:
a=nerdy('10.1037//0735-7044.112.6.1291')
print(a.paper_url)
a.get_full_text()
# html = urllib.request.urlopen(a.paper_url)`

c
http://dx.doi.org/10.1037//0735-7044.112.6.1291
link Requested Range Not Satisfiable


In [46]:
a.random_ten

array(['10.1111/j.1460-9568.2004.03710.x', '10.1016/j.neuron.2009.08.011',
       '10.1093/brain/awp090', '10.1037/a0012895',
       '10.1162/jocn.2006.18.9.1586', '10.1016/j.neuron.2009.08.011',
       '10.1002/hipo.20109', '10.1037/a0012895',
       '10.1016/j.neuropsychologia.2010.01.001',
       '10.1162/089892904322926692'], 
      dtype='<U63')

In [176]:
for link in a.citations:
    print(link.find_all('a').select('href'))

AttributeError: 'ResultSet' object has no attribute 'select'

In [146]:
a = nerdy('http://www.nature.com/nrn/journal/v13/n10/full/nrn3338.html')
a.get_full_text()

Nature Journal


In [152]:
a.citations[1].a['href']

'/doifinder/10.1038/35049064'

In [ ]:
html=urllib.request.urlopen('http://dx.doi.org/10.1038/35049064')
soup = bs.BeautifulSoup(html,'lxml')# trun pretty 
## extract
citations = soup.find_all('ol', attrs={'class':'references'})

In [ ]:
a = nerdy('http://www.nature.com/nrn/journal/v2/n1/full/nrn0101_051a.html')
a.get_full_text()
a.citations
a.citations[1].a['href']

In [ ]:
# a = nerdy('http://www.nature.com/nrn/journal/v13/n10/full/nrn3338.html')
a = nerdy('http://www.nature.com/nrn/journal/v13/n10/full/nrn3338.html') # nature neuro
a.get_full_text()
a.get_links()


In [ ]:
a = nerdy('http://www.jneurosci.org/content/32/19/6550')
a.get_full_text()

In [50]:
import networkx as nx

In [51]:
# this is where the magic happens
def recursive_add(DOI):
    if not DG.has_node( DOI ):
        node_data = nerdy(DOI)
        print(node_data.paper_url)
#         node_data.get_full_text()
#         node_data.get_links()
        node_data.get_full_text()

#         node_data.elsevier_ref()    
        DG.add_node( DOI)
        child_list = node_data.random_ten
#         return(child_list)
        print(DOI,child_list)
#     for child in child_list:
#         print(child)
#         recursive_add(child)
#         print( 'addding: '+child )
#         DG.add_edge(DOI, child)
    

In [52]:
DG = nx.DiGraph()
recursive_add('https://www.nature.com/neuro/journal/vaop/ncurrent/full/nn.4572.html')

b
https://www.nature.com/neuro/journal/vaop/ncurrent/full/nn.4572.html
Nature Journal
0 long link
1 long link
2 long link
3 long link
4 long link
5 long link
6 long link
7 long link
8 long link
9 long link
10 long link
11 long link
12 long link
13 long link
14 long link
15 long link
16 long link
17 long link
18 long link
19 long link
20 long link
21 long link
22 long link
23 long link
24 long link
25 long link
26 long link
27 long link
28 long link
29 long link
30 long link
31 long link
32 long link
33 long link
34 long link
35 long link
36 long link
37 long link
38 long link
39 long link
40 long link
41 long link
42 long link
43 long link
44 long link
45 long link
46 long link
https://www.nature.com/neuro/journal/vaop/ncurrent/full/nn.4572.html ['http://www.mathworks.com/matlabcentral/fileexchange/8998'
 'http://www.mathworks.com/matlabcentral/fileexchange/8998'
 'http://www.mathworks.com/matlabcentral/fileexchange/8998'
 'http://www.mathworks.com/matlabcentral/fileexchange/8998'
 'ht

In [ ]:
b = nerdy(a.random_ten[2])

In [ ]:
urllib.request.urlopen(b.paper_url)

In [ ]:
b.get_full_text()

In [ ]:
b.

In [ ]:
b.soup

In [ ]:
# b.get_full_text()
b.soup

In [ ]:
recursive_add(a.random_ten[1])